# Sonecule Tests

Notebook for testing the sonecules package.

In [ ]:
%load_ext autoreload
%autoreload 2

## Startup

In [ ]:
import sonecules as sn

In [ ]:
sn.startup()
sn.pb = sn.playback
ctx = sn.gcc()

In [ ]:
# sn.stop()
# ctx.close()

## Data Preparation

In [ ]:
%run prepare-data.ipynb

## Buffer Synthesis

In [ ]:
from sonecules.buffersyn import Audification

In [ ]:
audification = Audification(data=eeg_data[:,[0,8]], sr=256)

In [ ]:
audification.synth

In [ ]:
audification.synth.params["out"].bounds = (0,2)

In [ ]:
audification.synth.params

In [ ]:
ctx.reset()
audification.schedule(0, params={"rate":5})

In [ ]:
ctx.timeline

In [ ]:
ctx.timeline.head

In [ ]:
audification.reset()

In [ ]:
ctx.timeline

In [ ]:
ctx.timeline.head

In [ ]:
ctx.timeline

In [ ]:
ctx.timeline.plot()

In [ ]:
sn.pb().start()

In [ ]:
with ctx.at(5): audification.synth.stop()

### Audification with time stretch and rate control 

In [ ]:
from pya import Asig
import pytsmod as tsm
import sc3nb as scn
import matplotlib.pyplot as plt
from ipywidgets import interactive

%matplotlib widget
plt.rcParams["figure.figsize"] = (8,3)

In [ ]:
def audify_basic(data, sr=None, rate_pre=1, stretch_pre=1, rate_pb=100, amp=0.4):
    ctx = sn.gcc()
    if type(data) == Asig:
        dasig = data
    else:
        if not sr:
            print("audify_basic: missing sr (if data is not an Asig)")
        dasig = Asig(data, sr=sr)
    
    # data preprocessing (resampling/stretching, later: filtering for BL audification
    dasig = dasig.resample(target_sr=dasig.sr, rate=rate_pre) 

    # perform time scale mod on resampled data
    dasig = Asig(tsm.wsola(dasig.sig.T, s=stretch_pre).T, sr=dasig.sr)
    # dasig = Asig(tsm.wsola(dasig.sig.T, s=stretch_pre, win_size=100, syn_hop_size=30).T, sr=dasig.sr)

    buf = ctx.buffers.from_data(dasig.sig, dasig.sr)
    synth = ctx.synths.from_buffer(buf, synth_name="playbuf", mutable=False)

    ctx.reset()
    with ctx.at(time=0):
        synth.start(amp=amp, rate=rate_pb)
    sn.pb().start()
    return dasig

In [ ]:
plt.figure()
data = Asig(eeg_data, sr=256)[{8:40}, [3,5]]
audify_basic(data.norm(0.5), rate_pre=1, stretch_pre=4, rate_pb=20, amp=0.3).plot(offset=1)
plt.tight_layout(); plt.legend();

In [ ]:
ctx.stop()

### Audification with Phasor, i.e. adjustable time point and loop-capable

In [ ]:
# control ressampling on raw data to asig
dasig = Asig(eeg_data[:,[5,13]], sr=256).resample(target_sr=256, rate=1)
# perform time scale mod on resampled data
# dasig = Asig(tsm.wsola(dasig.sig.T, s=10).T, sr=dasig.sr)
# dasig = Asig(tsm.wsola(dasig.sig.T, s=10, win_size=100, syn_hop_size=30).T, sr=dasig.sr)

# put processed signal into buffer
buf = sn.gcc().buffers.from_data(dasig.sig, dasig.sr)

In [ ]:
# play at target rate by fast playback on playbuf
synth = sn.gcc().synths.from_buffer(buf, synth_name="playbuf", mutable=False)
sn.gcc().reset()
with sn.gcc().at(time=0):
    synth.start(amp=0.1, rate=100)
sn.playback().start()
# sn.gcc().timeline.plot()

In [ ]:
scn.SynthDef("sphasebuf3", 
"""{ 
| out=0, bufnum=0, trig=1, rate=1, relstart=0, relend=1, respos=0, amp=0.1 | 
    var framesInBuffer;
    framesInBuffer = BufFrames.kr(bufnum);
    //  Phasor.ar(trig, rate,                    start, end,            resetPos)
    //x = Phasor.ar(trig, rate*BufRateScale.kr(bufnum), relstart*framesInBuffer, relend*framesInBuffer, [0, MouseY.kr(0, framesInBuffer)]);
    x = Phasor.ar(trig, rate*BufRateScale.kr(bufnum), relstart*framesInBuffer, relend*framesInBuffer, respos*framesInBuffer);
    Out.ar(out, amp*BufRd.ar(1, bufnum, x));
}""").add();

In [ ]:
buf.frame_count

In [ ]:
# scn.Synth('sphasebuf')
st = sn.gcc().synths.create(name="sphasebuf3", track=1)

In [ ]:
bufnum = ctx.managers['buffers']._buffers[buf].bufnum
bufnum

In [ ]:
ctx = sn.gcc()
ctx.enable_realtime();
ctx.reset()

In [ ]:
st.start(bufnum=bufnum, rate=10, relstart=0.2, relend=0.4)

In [ ]:
def audify_gui(rate=20, amp=0.1, relstart=0.0, relwid=0, relend=1.0, respos=0):
    global st
    st.rate = rate
    st.amp = amp
    st.relstart = relstart
    if relwid==0:
        st.relend = relend
    else:
        st.relend = min(relstart + relwid, 1)
        
interactive(audify_gui, rate=(1, 100, 1), amp=(0,1,0.02), relstart=(0, 1, 0.001), relwid=(0,1,0.01), relend=(0,1,0.01), respos=(0,1,0.1))

In [ ]:
st.trig=1

In [ ]:
st.stop()

In [ ]:
ctx.disable_realtime()

In [ ]:
ctx.backend.sc.server.free_all()

In [ ]:
# sn.playback().time = 0 
# sn.playback().rate = 2

In [ ]:
# sn.playback().stop()  # stops playback (triggering of the scheduled events) but started synths will keep playing til finished

In [ ]:
sn.reset()

In [ ]:
sn.stop()  # stops playback and stops backend

## TimbralSonification

In [ ]:
timbralson = sn.buffersyn.TimbralSon(eeg_data[14*256:24*256], sr=256)

In [ ]:
timbralson.schedule(at=2, params={"amp": 0.03})    

In [ ]:
sn.playback().start()

In [ ]:
sn.stop()  # stops playback and stops backend

## ParameterSonification as Handler/Callback/Setup

In [ ]:
from sonecules.scoresyn import StandardContinuousPMSon, StandardDiscretePMSon

In [ ]:
from sc3nb import midicps, linlin

In [ ]:
penguins_df

In [ ]:
# adapting linlin to take dmin dmax as params - names open for discussion
dlinlin = lambda value, dmin, dmax, y1, y2: linlin(value, x1=dmin, x2=dmax, y1=y1, y2=y2)

In [ ]:
scpmson = StandardContinuousPMSon("s2", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)

test_mapping = {
    "onset": ("body_mass_g", dlinlin, {"y1": 0, "y2": 3}),
    "freq" : ("flipper_length_mm", dlinlin, {"y1": midicps(70), "y2": midicps(50)})
}

sn.reset() # needed for now as sonecule.reset() does not work yet
scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 


In [ ]:
sn.playback().start()

In [ ]:
# test_mapping = {"onset": (col, fun, mkwargs)}  

# ditched conversion for now

scpmson = StandardDiscretePMSon("s1", 
    {"freq": {"bounds": (midicps(49.9), midicps(70.1))},  # bounds checking in mesonic is too strict currently
     "amp": {"default": 0.1}}
)  # bounds are in pre conversion unit - but this is done differently here in the code

sn.reset() # needed for now as sonecule.reset() does not work yet
scpmson.schedule(df=penguins_df, mapping=test_mapping, at=0, stop_after=0.2)
# Error message when bounds are not respected by the mapping are currently quite hard to understand
# should fix this in mesonic 


In [ ]:
sn.playback().start()

In [ ]:
sn.playback().time = 1.5

In [ ]:
sn.stop()

## Sonification as Handler/Callback/Setup

In [ ]:
from sonecules.triggersyn import DataSonogram

In [ ]:
%matplotlib qt

In [ ]:
sn.reset()
sn.gcc().enable_realtime()

In [ ]:
dsg1 = DataSonogram(penguins_df, x="flipper_length_mm", y="body_mass_g", label="species")